In [ ]:
current_book = [ 
    # [bid_size, bid, ask, ask_size]
    [1, 100, 105, 2],
    [1, 98, 106, 5]
]
new_orders = [
    # [side (buy=1, sell=2), size, price, orderId] 
    [1, 1, 100, 'aa1'],
    [1, 1, 95, 'aa2'],
    [1, 5, 98, 'bb1'],
    [2, 7, 96, 'cc1'],
    [2, -7, 96, 'cc1'],
    [2, 1, 98, 'aa3'],
    [1, 5, 100, 'dd1'],
    [2, 5, 101, 'dd2'],
    [1, 1, 105, 'aa4'],
    [1, -1, 100, 'aa1'],
]

# we are account "aa"

sorted_orders = [
    # [side (buy=1, sell=2), size, price, orderId] 
    [1, 1, 100, 'aa1'],
    [1, -1, 100, 'aa1'],
    [1, 5, 100, 'dd1'],
    [2, 1, 98, 'aa3'],
    [1, 1, 95, 'aa2'],
    [1, 5, 98, 'bb1'],
    [2, 7, 96, 'cc1'],
    [2, -7, 96, 'cc1'],
    [2, 5, 101, 'dd2'],
    [1, 1, 105, 'aa4'],
]

In [ ]:
import threading
from hashlib import sha256
import sys
import os

global kill # variable to track when a suitable nonce is found and kill all threads
kill = False

def find_nonce(control, array, range_low, range_high):
    global kill
    
    for nonce in range(range_low, range_high):
        if kill == True:
            break
            
        l = []
        for i in array:
            input_ = f'{control} {nonce} {i}'
            l.append(sha256(input_.encode('utf-8')).hexdigest())
        if l == sorted(l): # sort hash encoded list and compare to our defined order
            print(nonce)
            kill = True

control = 0

threads = []
no_threads = 10 # also used to split the brute force range into smaller chunks to run on multiple threads
for i in range(no_threads):
    thread = threading.Thread(
        target=find_nonce,
        args=(
            control, sorted_orders, int(sys.maxsize/no_threads)*i, int(sys.maxsize/no_threads)*(i+1),
        ),
        daemon=True
    )
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()